# CSF/PET Baseline Testing Verification

This notebook verifies that the CSF/PET baseline testing pathway in `components/testing.py`
works as intended. CSF/PET testing is distinct from BBBM testing and operates across all
three scenarios.

### How CSF/PET testing works (`_update_baseline_testing`)

- **Eligibility**: simulant must be in MCI or Dementia state, with
  `testing_propensity < (csf_rate + pet_rate)`, NOT already CSF/PET tested,
  and NOT BBBM-positive
- **Rates**: loaded from artifact per draw (draw 0 for this sim); population means
  are CSF~10.8%, PET~15.0% but actual draw values differ
- **Test type assignment**: weighted random choice between CSF and PET proportional
  to their rates (vivarium normalizes the weights internally)
- **One-time**: once a simulant gets CSF or PET, they stay in that state permanently
- **All scenarios**: CSF/PET runs in baseline, bbbm_testing, and bbbm_testing_and_treatment

### Checks

| # | Check | Type |
|---|-------|------|
| 1 | Only MCI/Dementia simulants get CSF/PET tests | Disease state gating |
| 2 | Propensity threshold is deterministic | Exact check |
| 3 | BBBM-positive blocks CSF/PET | Interaction with BBBM |
| 4 | CSF/PET split matches rate ratio | Statistical |
| 5 | Testing fraction among eligible matches combined rate | Statistical |
| 6 | CSF/PET works identically across scenarios (modulo BBBM blocking) | Cross-scenario |

### How to use

Run all cells. Assert statements at the end confirm key invariants.
If the notebook completes without error, all checks pass.

In [1]:
import math

import pandas as pd
import numpy as np
from loguru import logger
from tqdm.auto import tqdm
from vivarium import InteractiveContext

# Suppress vivarium's verbose logging
logger.disable("vivarium")
logger.disable("vivarium_public_health")
logger.disable("vivarium_csu_alzheimers")

SPEC_PATH = '../src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml'
POPULATION_SIZE = 10_000
STEP_SIZE_DAYS = 182
TARGET_YEAR = 2050

# Disease states
BBBM_STATE = 'alzheimers_blood_based_biomarker_state'
MCI_STATE = 'alzheimers_mild_cognitive_impairment_state'
DEMENTIA_STATE = 'alzheimers_disease_state'
DISEASE_COL = 'alzheimers_disease_and_other_dementias'

# Load actual testing rates from artifact (draw 0, matching input_draw_number in spec)
ARTIFACT_PATH = '../united_states_of_america.hdf'
DRAW = 0
with pd.HDFStore(ARTIFACT_PATH, mode='r') as store:
    CSF_RATE = float(store['/testing_rates/csf'][f'draw_{DRAW}'].iloc[0])
    PET_RATE = float(store['/testing_rates/pet'][f'draw_{DRAW}'].iloc[0])

COMBINED_RATE = CSF_RATE + PET_RATE
EXPECTED_CSF_FRAC = CSF_RATE / COMBINED_RATE
EXPECTED_PET_FRAC = PET_RATE / COMBINED_RATE

print(f'CSF rate (draw {DRAW}): {CSF_RATE:.6f}')
print(f'PET rate (draw {DRAW}): {PET_RATE:.6f}')
print(f'Combined threshold: {COMBINED_RATE:.6f}')
print(f'Expected CSF fraction: {EXPECTED_CSF_FRAC:.4f}')
print(f'Expected PET fraction: {EXPECTED_PET_FRAC:.4f}')

CSF rate (draw 0): 0.086170
PET rate (draw 0): 0.172089
Combined threshold: 0.258259
Expected CSF fraction: 0.3337
Expected PET fraction: 0.6663


In [2]:
def step_to_year(sim, target_year, target_month=7):
    """Step the simulation forward with a progress bar."""
    target = pd.Timestamp(f'{target_year}-{target_month:02d}-01')
    est_steps = max(1, math.ceil((target - sim.current_time).days / STEP_SIZE_DAYS))
    steps = 0
    with tqdm(total=est_steps, desc=f'-> {target_year}', unit='step') as pbar:
        while sim.current_time < target:
            sim.step()
            steps += 1
            pbar.update(1)
        pbar.total = steps
        pbar.refresh()
    return sim

---
## Run baseline scenario to 2050

We need enough time for simulants to progress through BBBM → MCI → Dementia and
accumulate meaningful CSF/PET testing counts.

In [3]:
sim_baseline = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'baseline'},
    }
)
print(f'Initialized at {sim_baseline.current_time}')
step_to_year(sim_baseline, TARGET_YEAR)
pop_bl = sim_baseline.get_population()
alive_bl = pop_bl[pop_bl['alive'] == 'alive'].copy()
print(f'\nAt {sim_baseline.current_time.date()}: {len(alive_bl)} alive simulants')
print(f'Disease states:\n{alive_bl[DISEASE_COL].value_counts().to_string()}')
print(f'\nTesting states:\n{alive_bl["testing_state"].value_counts().to_string()}')

/home/abie/miniforge3/envs/vivarium_csu_alzheimers_simulation/lib/python3.11/site-packages/vivarium_public_health/plugins/parser.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Initialized at 2022-01-01 00:00:00


-> 2050:   0%|          | 0/58 [00:00<?, ?step/s]


At 2050-11-26: 20256 alive simulants
Disease states:
alzheimers_disease_and_other_dementias
alzheimers_blood_based_biomarker_state        9407
alzheimers_disease_state                      6571
alzheimers_mild_cognitive_impairment_state    4278

Testing states:
testing_state
not_tested    17510
pet            1847
csf             899


---
## Check 1: Only MCI/Dementia simulants get CSF/PET tests

No simulant in BBBM state (or susceptible, if any existed) should have
`testing_state` of `csf` or `pet`.

In [4]:
csf_pet_tested = alive_bl[alive_bl['testing_state'].isin(['csf', 'pet'])]

disease_states_of_tested = csf_pet_tested[DISEASE_COL].value_counts()
print('Disease states of CSF/PET tested simulants:')
print(disease_states_of_tested.to_string())

# Check: every CSF/PET tested simulant is in MCI or Dementia
# Note: we check current state. A simulant tested while in MCI may have
# since progressed to Dementia, so both are valid.
invalid_states = csf_pet_tested[~csf_pet_tested[DISEASE_COL].isin([MCI_STATE, DEMENTIA_STATE])]
n_invalid = len(invalid_states)
print(f'\nCSF/PET tested in non-MCI/Dementia state: {n_invalid}')

# In baseline, there's no BBBM testing, so testing_state can only be
# 'not_tested', 'csf', or 'pet'. A simulant in BBBM disease state
# hasn't reached MCI yet, so should never be CSF/PET tested.
bbbm_with_csf_pet = alive_bl[
    (alive_bl[DISEASE_COL] == BBBM_STATE)
    & alive_bl['testing_state'].isin(['csf', 'pet'])
]
print(f'BBBM-state simulants with CSF/PET: {len(bbbm_with_csf_pet)} (should be 0)')

Disease states of CSF/PET tested simulants:
alzheimers_disease_and_other_dementias
alzheimers_disease_state                      1690
alzheimers_mild_cognitive_impairment_state    1056

CSF/PET tested in non-MCI/Dementia state: 0
BBBM-state simulants with CSF/PET: 0 (should be 0)


---
## Check 2: Propensity gating works correctly

All CSF/PET tested simulants should have `testing_propensity < combined_rate`.
Among untested MCI/Dementia simulants (non-BBBM-positive), none should have
`testing_propensity < combined_rate` — because baseline testing is applied every step,
so any eligible simulant below the threshold would have been tested immediately.

In the baseline scenario there's no BBBM testing, so `bbbm_test_result` is
always `not_tested` and doesn't block anyone.

**Note**: The actual rates come from the artifact (draw-specific), not the population
means. This is critical for exact propensity checks.

In [5]:
# All CSF/PET tested should be below threshold
tested_above = csf_pet_tested[csf_pet_tested['testing_propensity'] >= COMBINED_RATE]
n_tested_above = len(tested_above)
print(f'CSF/PET tested with propensity >= {COMBINED_RATE}: {n_tested_above} (should be 0)')

# Among MCI/Dementia simulants who are NOT CSF/PET tested, none should be below threshold
# (In baseline, bbbm_test_result is always 'not_tested', so no BBBM-positive blocking)
mci_dementia = alive_bl[alive_bl[DISEASE_COL].isin([MCI_STATE, DEMENTIA_STATE])]
untested_mci_dem = mci_dementia[~mci_dementia['testing_state'].isin(['csf', 'pet'])]
untested_below = untested_mci_dem[untested_mci_dem['testing_propensity'] < COMBINED_RATE]
n_untested_below = len(untested_below)
print(f'Untested MCI/Dementia with propensity < {COMBINED_RATE}: {n_untested_below} (should be 0)')

if n_untested_below > 0:
    print('\nInvestigating untested below-threshold simulants...')
    print(untested_below[['testing_state', 'testing_propensity', 'bbbm_test_result',
                          DISEASE_COL]].head(10))

CSF/PET tested with propensity >= 0.25825909497717264: 0 (should be 0)
Untested MCI/Dementia with propensity < 0.25825909497717264: 0 (should be 0)


---
## Check 4: CSF/PET split matches rate ratio

Among CSF/PET tested simulants, the proportion should match the rate ratio:
- CSF: csf_rate / (csf_rate + pet_rate)
- PET: pet_rate / (csf_rate + pet_rate)

The `randomness.choice` uses unnormalized weights `[csf_rate, pet_rate]` which vivarium
normalizes internally.

In [6]:
n_csf = int((csf_pet_tested['testing_state'] == 'csf').sum())
n_pet = int((csf_pet_tested['testing_state'] == 'pet').sum())
n_total = n_csf + n_pet

if n_total > 0:
    actual_csf_frac = n_csf / n_total
    actual_pet_frac = n_pet / n_total
else:
    actual_csf_frac = actual_pet_frac = 0.0

print(f'CSF/PET tested: {n_total} total')
print(f'  CSF: {n_csf} ({actual_csf_frac:.1%})  expected ~{EXPECTED_CSF_FRAC:.1%}')
print(f'  PET: {n_pet} ({actual_pet_frac:.1%})  expected ~{EXPECTED_PET_FRAC:.1%}')
print(f'\n  CSF deviation: {abs(actual_csf_frac - EXPECTED_CSF_FRAC):.2%} from expected')
print(f'  PET deviation: {abs(actual_pet_frac - EXPECTED_PET_FRAC):.2%} from expected')

CSF/PET tested: 2746 total
  CSF: 899 (32.7%)  expected ~33.4%
  PET: 1847 (67.3%)  expected ~66.6%

  CSF deviation: 0.63% from expected
  PET deviation: 0.63% from expected


---
## Check 5: Testing fraction among eligible

Among MCI/Dementia simulants, approximately `combined_rate` fraction should have
`testing_propensity < threshold` and thus eventually be tested. The propensity is
a uniform [0, 1) draw, so the fraction below the threshold should be close to
the combined rate.

In [7]:
# Among all MCI/Dementia simulants (alive), what fraction has propensity < threshold?
n_mci_dem = len(mci_dementia)
n_below_threshold = int((mci_dementia['testing_propensity'] < COMBINED_RATE).sum())
frac_below = n_below_threshold / max(n_mci_dem, 1)

print(f'MCI/Dementia alive: {n_mci_dem}')
print(f'Propensity < {COMBINED_RATE}: {n_below_threshold} ({frac_below:.1%})')
print(f'Expected: ~{COMBINED_RATE:.1%}')
print(f'Deviation: {abs(frac_below - COMBINED_RATE):.2%}')

# Also verify: among CSF/PET tested, count should match n_below_threshold
# (in baseline, no BBBM-positive blocking, so all below-threshold MCI/Dementia
# should be tested)
n_csf_pet_in_mci_dem = int(
    mci_dementia['testing_state'].isin(['csf', 'pet']).sum()
)
print(f'\nActually CSF/PET tested among MCI/Dementia: {n_csf_pet_in_mci_dem}')
print(f'Below-threshold MCI/Dementia: {n_below_threshold}')
print(f'Match: {n_csf_pet_in_mci_dem == n_below_threshold}')

MCI/Dementia alive: 10849
Propensity < 0.25825909497717264: 2746 (25.3%)
Expected: ~25.8%
Deviation: 0.51%

Actually CSF/PET tested among MCI/Dementia: 2746
Below-threshold MCI/Dementia: 2746
Match: True


---
## Run bbbm_testing scenario to 2050

This scenario adds BBBM testing on top of baseline CSF/PET testing.
BBBM-positive simulants should be blocked from CSF/PET.

In [8]:
sim_bbbm = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'bbbm_testing'},
    }
)
print(f'Initialized at {sim_bbbm.current_time}')
step_to_year(sim_bbbm, TARGET_YEAR)
pop_bbbm = sim_bbbm.get_population()
alive_bbbm = pop_bbbm[pop_bbbm['alive'] == 'alive'].copy()
print(f'\nAt {sim_bbbm.current_time.date()}: {len(alive_bbbm)} alive simulants')
print(f'Disease states:\n{alive_bbbm[DISEASE_COL].value_counts().to_string()}')
print(f'\nTesting states:\n{alive_bbbm["testing_state"].value_counts().to_string()}')
print(f'\nBBBM test results:\n{alive_bbbm["bbbm_test_result"].value_counts().to_string()}')

Initialized at 2022-01-01 00:00:00


-> 2050:   0%|          | 0/58 [00:00<?, ?step/s]


At 2050-11-26: 20256 alive simulants
Disease states:
alzheimers_disease_and_other_dementias
alzheimers_blood_based_biomarker_state        9407
alzheimers_disease_state                      6571
alzheimers_mild_cognitive_impairment_state    4278

Testing states:
testing_state
not_tested    14381
bbbm           3889
pet            1332
csf             654

BBBM test results:
bbbm_test_result
not_tested    15849
positive       2575
negative       1832


---
## Check 3: BBBM-positive blocks CSF/PET

In the bbbm_testing scenario, no simulant with `bbbm_test_result == 'positive'`
should have `testing_state` of `csf` or `pet`.

The blocking logic is in `_update_baseline_testing` line 132:
```python
eligible_bbbm_results = pop[COLUMNS.BBBM_TEST_RESULT] != BBBM_TEST_RESULTS.POSITIVE
```

In [9]:
bbbm_positive = alive_bbbm[alive_bbbm['bbbm_test_result'] == 'positive']
positive_with_csf_pet = bbbm_positive[bbbm_positive['testing_state'].isin(['csf', 'pet'])]
n_positive_with_csf_pet = len(positive_with_csf_pet)

print(f'BBBM-positive simulants: {len(bbbm_positive)}')
print(f'BBBM-positive with CSF/PET: {n_positive_with_csf_pet} (should be 0)')

# Also check: BBBM-positive simulants should have testing_state of 'bbbm' or 'not_tested'
positive_testing_states = bbbm_positive['testing_state'].value_counts()
print(f'\nTesting states among BBBM-positive:\n{positive_testing_states.to_string()}')

BBBM-positive simulants: 2575
BBBM-positive with CSF/PET: 0 (should be 0)

Testing states among BBBM-positive:
testing_state
bbbm    2575


---
## Check 6: Cross-scenario comparison

CSF/PET testing should work identically in baseline and bbbm_testing scenarios,
with the only difference being that BBBM-positive simulants are blocked from
CSF/PET in the bbbm_testing scenario.

Due to CRN (common random numbers), simulants with the same entrance time and age
get the same propensity draws. So the CSF/PET split ratio should be very similar
across scenarios.

In [10]:
# Compare CSF/PET counts
bl_csf = int((alive_bl['testing_state'] == 'csf').sum())
bl_pet = int((alive_bl['testing_state'] == 'pet').sum())
bl_total = bl_csf + bl_pet

bbbm_csf = int((alive_bbbm['testing_state'] == 'csf').sum())
bbbm_pet = int((alive_bbbm['testing_state'] == 'pet').sum())
bbbm_total = bbbm_csf + bbbm_pet

print(f'{"":<20} {"Baseline":>10} {"BBBM Testing":>14} {"Diff":>8}')
print('-' * 55)
print(f'{"CSF tested":<20} {bl_csf:>10} {bbbm_csf:>14} {bbbm_csf - bl_csf:>+8}')
print(f'{"PET tested":<20} {bl_pet:>10} {bbbm_pet:>14} {bbbm_pet - bl_pet:>+8}')
print(f'{"Total CSF/PET":<20} {bl_total:>10} {bbbm_total:>14} {bbbm_total - bl_total:>+8}')

# The bbbm_testing scenario should have FEWER CSF/PET tested because
# BBBM-positive simulants are blocked
print(f'\nBBBM testing has fewer CSF/PET: {bbbm_total <= bl_total}')
print(f'Difference: {bl_total - bbbm_total} fewer in BBBM scenario')

# Compare CSF/PET split ratios
bl_csf_frac = bl_csf / max(bl_total, 1)
bbbm_csf_frac = bbbm_csf / max(bbbm_total, 1)
print(f'\nCSF fraction - Baseline: {bl_csf_frac:.3f}, BBBM Testing: {bbbm_csf_frac:.3f}')
print(f'Split ratio difference: {abs(bl_csf_frac - bbbm_csf_frac):.3f}')

                       Baseline   BBBM Testing     Diff
-------------------------------------------------------
CSF tested                  899            654     -245
PET tested                 1847           1332     -515
Total CSF/PET              2746           1986     -760

BBBM testing has fewer CSF/PET: True
Difference: 760 fewer in BBBM scenario

CSF fraction - Baseline: 0.327, BBBM Testing: 0.329
Split ratio difference: 0.002


---
## Assertions

Automated checks for all 6 verification points.

In [11]:
results = []

# Check 1: Only MCI/Dementia get CSF/PET
assert len(bbbm_with_csf_pet) == 0, (
    f'{len(bbbm_with_csf_pet)} BBBM-state simulants have CSF/PET testing')
# n_invalid counts tested simulants NOT in MCI or Dementia.
# Because testing is one-time and disease only progresses forward
# (MCI -> Dementia), all tested simulants should be in MCI or Dementia.
assert n_invalid == 0, (
    f'{n_invalid} CSF/PET tested simulants in non-MCI/Dementia state')
results.append(('Check 1: Only MCI/Dementia get CSF/PET', 'PASS'))
print('PASS: Check 1 - Only MCI/Dementia simulants get CSF/PET tests')

PASS: Check 1 - Only MCI/Dementia simulants get CSF/PET tests


In [12]:
# Check 2: Propensity gating is exact
assert n_tested_above == 0, (
    f'{n_tested_above} CSF/PET tested simulants have propensity >= {COMBINED_RATE}')
assert n_untested_below == 0, (
    f'{n_untested_below} untested MCI/Dementia simulants have propensity < {COMBINED_RATE}')
results.append(('Check 2: Propensity gating is exact', 'PASS'))
print('PASS: Check 2 - Propensity threshold deterministically gates CSF/PET testing')

PASS: Check 2 - Propensity threshold deterministically gates CSF/PET testing


In [13]:
# Check 3: BBBM-positive blocks CSF/PET
assert n_positive_with_csf_pet == 0, (
    f'{n_positive_with_csf_pet} BBBM-positive simulants have CSF/PET testing')
results.append(('Check 3: BBBM-positive blocks CSF/PET', 'PASS'))
print('PASS: Check 3 - No BBBM-positive simulant has CSF/PET testing')

PASS: Check 3 - No BBBM-positive simulant has CSF/PET testing


In [14]:
# Check 4: CSF/PET split matches expected ratio (within 5pp)
assert n_total > 100, f'Too few CSF/PET tested ({n_total}) for meaningful split check'
assert abs(actual_csf_frac - EXPECTED_CSF_FRAC) < 0.05, (
    f'CSF fraction {actual_csf_frac:.3f} too far from expected {EXPECTED_CSF_FRAC:.3f}')
assert abs(actual_pet_frac - EXPECTED_PET_FRAC) < 0.05, (
    f'PET fraction {actual_pet_frac:.3f} too far from expected {EXPECTED_PET_FRAC:.3f}')
results.append(('Check 4: CSF/PET split matches rate ratio', 'PASS'))
print(f'PASS: Check 4 - CSF/PET split ({actual_csf_frac:.1%}/{actual_pet_frac:.1%}) '
      f'within 5pp of expected ({EXPECTED_CSF_FRAC:.1%}/{EXPECTED_PET_FRAC:.1%})')

PASS: Check 4 - CSF/PET split (32.7%/67.3%) within 5pp of expected (33.4%/66.6%)


In [15]:
# Check 5: Testing fraction among eligible ~25.8%
assert n_mci_dem > 100, f'Too few MCI/Dementia simulants ({n_mci_dem})'
assert abs(frac_below - COMBINED_RATE) < 0.05, (
    f'Testing fraction {frac_below:.3f} too far from expected {COMBINED_RATE:.3f}')
# In baseline (no BBBM blocking), tested count should equal below-threshold count
assert n_csf_pet_in_mci_dem == n_below_threshold, (
    f'Tested ({n_csf_pet_in_mci_dem}) != below-threshold ({n_below_threshold})')
results.append(('Check 5: Testing fraction ~25.8%', 'PASS'))
print(f'PASS: Check 5 - Testing fraction {frac_below:.1%} close to expected {COMBINED_RATE:.1%}')

PASS: Check 5 - Testing fraction 25.3% close to expected 25.8%


In [16]:
# Check 6: Cross-scenario comparison
# BBBM testing scenario should have fewer or equal CSF/PET (BBBM-positive blocking)
assert bbbm_total <= bl_total, (
    f'BBBM scenario has MORE CSF/PET ({bbbm_total}) than baseline ({bl_total})')
# CSF/PET split ratio should be similar across scenarios (within 5pp)
assert abs(bl_csf_frac - bbbm_csf_frac) < 0.05, (
    f'CSF split differs: baseline {bl_csf_frac:.3f} vs BBBM {bbbm_csf_frac:.3f}')
results.append(('Check 6: Cross-scenario comparison', 'PASS'))
print(f'PASS: Check 6 - BBBM scenario has {bl_total - bbbm_total} fewer CSF/PET '
      f'(blocking), split ratio within 5pp')

PASS: Check 6 - BBBM scenario has 760 fewer CSF/PET (blocking), split ratio within 5pp


In [17]:
print('\n' + '=' * 50)
print('SUMMARY')
print('=' * 50)
for name, status in results:
    print(f'  {status}: {name}')
print(f'\n=== ALL {len(results)} CHECKS PASSED ===')


SUMMARY
  PASS: Check 1: Only MCI/Dementia get CSF/PET
  PASS: Check 2: Propensity gating is exact
  PASS: Check 3: BBBM-positive blocks CSF/PET
  PASS: Check 4: CSF/PET split matches rate ratio
  PASS: Check 5: Testing fraction ~25.8%
  PASS: Check 6: Cross-scenario comparison

=== ALL 6 CHECKS PASSED ===
